In [49]:
from azure.common.credentials import ServicePrincipalCredentials
from azure.mgmt.resource import ResourceManagementClient
from azure.mgmt.datafactory import DataFactoryManagementClient
from azure.mgmt.datafactory.models import *
from datetime import datetime, timedelta
import time
import json
from mockODBCData import get_metadata,replicate_sap_source,create_query

In [50]:
print(schema)

_SYS_BIC


In [51]:
def make_source_query(mapping:dict,source_name:str,schema:str) -> str:
    """
    """
    source_name = schema + '.' + source_name
    return f'SELECT {",".join(mapping.keys())} from {source_name}'

def make_precopy_script(sql_table:str) -> str:
    """  
    """
    return f'TRUNCATE TABLE [dbo].{sql_table}'

In [52]:
#schema = "_SYS_BIC"
#sourcename = "VBAP_DEMO"
#source_type = "VIEW"
def get_default_mapping(schema,sourcename,istable):
    if istable:
        source_type = "TABLE"
    else:
        source_type = "VIEW"
    df = get_metadata(schema=schema,sourcename=sourcename,source_type=source_type) 
    #df has sap metadeta
    return { x:x for x in list(df.index)}

In [53]:
#SAPHANADB-schema
#VBAP-sourcename
#True-use_default_mapping
def make_translator(schema,sourcename,use_default_mapping = False,istable=False)->TabularTranslator:
    """
    Creates TabularTranslator using given mapping or default mapping 
  
    Parameters: 
       schema (string): 
       viewname (string):
    """
    if use_default_mapping:
        source = get_default_mapping(schema=schema,sourcename=sourcename,istable=istable)
    else:
        source = inputs['mapping'] 
        
    mappings = []
    
###############################    
    for key in source:
        obj = {}
        obj['source'] = {"name":key}
        obj['sink'] = {"name":source[key]} #Stripping square braces
        mappings.append(obj)
    return TabularTranslator.from_dict({"type":"TabularTranslator","mappings":mappings})

### Execution

In [59]:
schema = "_SYS_BIC"
source = "Temp/VBAP_DEMO"
istable = False
source_type = "VIEW"
sql_table = source

In [60]:
inputs = json.load(open('input.json'))


In [61]:
mapping = get_default_mapping(schema=schema,sourcename=source,istable=istable)

In [62]:
print(mapping)

{'ABDAT': 'ABDAT', 'ABFOR': 'ABFOR', 'ABGES': 'ABGES', 'ABGRS': 'ABGRS', 'ABGRU': 'ABGRU', 'ABLFZ': 'ABLFZ', 'ABSFZ': 'ABSFZ', 'ABSTA': 'ABSTA', 'AD01PROFNR': 'AD01PROFNR', 'ADACN': 'ADACN', 'ADDNS': 'ADDNS', 'ADICC': 'ADICC', 'ADMOI': 'ADMOI', 'ADPRI': 'ADPRI', 'ADPROF': 'ADPROF', 'ADPROF_1': 'ADPROF_1', 'ADSPC_SPC': 'ADSPC_SPC', 'ADSPC_SPC_1': 'ADSPC_SPC_1', 'AEDAT': 'AEDAT', 'AEKLK': 'AEKLK', 'AEKLK_1': 'AEKLK_1', 'AENAM': 'AENAM', 'AENAM_1': 'AENAM_1', 'AESZN': 'AESZN', 'AESZN_1': 'AESZN_1', 'ALLOW_PMAT_IGNO': 'ALLOW_PMAT_IGNO', 'ALLOW_PMAT_IGNO_1': 'ALLOW_PMAT_IGNO_1', 'ANIMAL_ORIGIN': 'ANIMAL_ORIGIN', 'ANIMAL_ORIGIN_1': 'ANIMAL_ORIGIN_1', 'ANP': 'ANP', 'ANP_1': 'ANP_1', 'ANTLF': 'ANTLF', 'ANZSN': 'ANZSN', 'APCGK_EXTENDI': 'APCGK_EXTENDI', 'APLZL_OAA': 'APLZL_OAA', 'APLZL_OLC': 'APLZL_OLC', 'ARKTX': 'ARKTX', 'ARSNUM': 'ARSNUM', 'ARSPOS': 'ARSPOS', 'ATPKZ': 'ATPKZ', 'ATP_ABC_SUBSTITUTION_STATUS': 'ATP_ABC_SUBSTITUTION_STATUS', 'ATTYP': 'ATTYP', 'ATTYP_1': 'ATTYP_1', 'AUFNR': 'AUFNR

In [32]:
subscription_id = '938ace66-9598-4029-b6bb-429929b03761'
rg_name = 'celebal_rnd'
df_name = 'celebaladf'
client_id = 'b628371b-654f-4848-b214-c8553f2fc665' 
secret = '/JCA4now2LAn1/L4aa+ICfmTumPRryW.'
tenant = 'e4e34038-ea1f-4882-b6e8-ccd776459ca0'
rg_params = {'location':'eastus'}
df_params = {'location':'eastus'}

In [63]:
try:
    replicate_sap_source(schema=schema,source=source,source_type=source_type)
except Exception as e:
    print(e)

Replicating SAP Source
Connecting to DB
Temp/VBAP_DEMO


In [64]:
# Specify your AD client ID, client secret, and tenant ID
credentials = ServicePrincipalCredentials(client_id=client_id, secret=secret, tenant=tenant)
resource_client = ResourceManagementClient(credentials, subscription_id)
adf_client = DataFactoryManagementClient(credentials, subscription_id)

In [65]:
ds_name = 'RelationalTable1'
dsOut_name = 'sap_hana_db'

In [66]:
source_ds = adf_client.datasets.get(dataset_name=ds_name,resource_group_name=rg_name,factory_name=df_name)
destination_ds = adf_client.datasets.get(dataset_name=dsOut_name,resource_group_name=rg_name,factory_name=df_name)

In [67]:
print(source_ds)

{'additional_properties': {}, 'id': '/subscriptions/938ace66-9598-4029-b6bb-429929b03761/resourceGroups/celebal_rnd/providers/Microsoft.DataFactory/factories/celebaladf/datasets/RelationalTable1', 'name': 'RelationalTable1', 'type': 'Microsoft.DataFactory/factories/datasets', 'etag': '0600e2b3-0000-0100-0000-5cebc8c20000', 'properties': <azure.mgmt.datafactory.models.relational_table_dataset.RelationalTableDataset object at 0x7fa1674ee198>}


In [68]:
print(destination_ds)

{'additional_properties': {}, 'id': '/subscriptions/938ace66-9598-4029-b6bb-429929b03761/resourceGroups/celebal_rnd/providers/Microsoft.DataFactory/factories/celebaladf/datasets/sap_hana_db', 'name': 'sap_hana_db', 'type': 'Microsoft.DataFactory/factories/datasets', 'etag': '0600e1b3-0000-0100-0000-5cebc8c20000', 'properties': <azure.mgmt.datafactory.models.azure_sql_table_dataset.AzureSqlTableDataset object at 0x7fa1674e5828>}


In [69]:
source_type = RelationalSource(query=make_source_query(mapping=mapping,source_name=source,schema=schema))#query to select columns
destination_type = SqlSink()
#pre_copy_script=make_precopy_script(sql_table)

In [72]:
dsin_ref = DatasetReference(ds_name)
dsOut_ref = DatasetReference(dsOut_name,parameters={
    "TABLENAME":f"[dbo].[{sql_table}]"
})

In [75]:
trans = make_translator(schema=schema,sourcename=source,istable=istable,use_default_mapping=True)

In [76]:
copy_activity = CopyActivity('SAP HANA TO SQL DBtest',inputs=[dsin_ref], outputs=[dsOut_ref],source=source_type, sink=destination_type,translator=trans)
p_name = 'saphanatest'
p_obj = PipelineResource(activities=[copy_activity])
p = adf_client.pipelines.create_or_update(rg_name, df_name, p_name, p_obj)